In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
#from datetime import timedelta  ## use this to increment targ by 1 year!!!!
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
#from patsy import dmatrices
#import random

import sys
import os
os.chdir('C:/BankRisk')

In [2]:
import time
from time import gmtime, strftime
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2016-05-31 06:02:12


In [3]:
dm = pd.read_csv("ladder_durmv_allmat_longhist.csv",header=[0,1],index_col=0,parse_dates=True) ## NOTE THIS COMMAND!!!
dm.head(2)

dv12                                                    \
                  ff        mv      mvpc       dur       cpn       mv3   
1965-12-01  4.380000  9.872790  1.008227  0.534467  5.493205  4.784516   
1966-01-01  4.481389  9.288667  1.007741  0.534508  5.463205  4.955346   

                               dv24       ...   dv300                 dv360  \
             roll12m    roll3m   mv mvpc  ...     mv3 roll300m roll3m    mv   
1965-12-01  4.854038  4.998342  NaN  NaN  ...     NaN      NaN    NaN   NaN   
1966-01-01  4.916538  5.187601  NaN  NaN  ...     NaN      NaN    NaN   NaN   

                                             
           mvpc dur cpn mv3 roll360m roll3m  
1965-12-01  NaN NaN NaN NaN      NaN    NaN  
1966-01-01  NaN NaN NaN NaN      NaN    NaN  

[2 rows x 106 columns]

In [5]:
dmq = dm.resample("Q").mean()
#dmq = dmq.dropna(how='any')
dmq.head(2)

dv12                                                   \
                  ff       mv      mvpc       dur       cpn       mv3   
1965-12-31  4.380000  9.87279  1.008227  0.534467  5.493205  4.784516   
1966-03-31  4.619954  9.60940  1.008008  0.534447  5.559872  5.331311   

                               dv24       ...   dv300                 dv360  \
             roll12m    roll3m   mv mvpc  ...     mv3 roll300m roll3m    mv   
1965-12-31  4.854038  4.998342  NaN  NaN  ...     NaN      NaN    NaN   NaN   
1966-03-31  4.986816  5.337432  NaN  NaN  ...     NaN      NaN    NaN   NaN   

                                             
           mvpc dur cpn mv3 roll360m roll3m  
1965-12-31  NaN NaN NaN NaN      NaN    NaN  
1966-03-31  NaN NaN NaN NaN      NaN    NaN  

[2 rows x 106 columns]

In [6]:
dmq120=pd.DataFrame(dmq['dv120']['roll120m'])
dmq144 = dmq['dv144']['roll144m']
dmq180 = dmq['dv180']['roll180m']
dmq240 = dmq['dv240']['roll240m']
dmq300 = dmq['dv300']['roll300m']
dmq360 = dmq['dv360']['roll360m']
dmx = pd.concat([dmq120,dmq144,dmq180,dmq240,dmq300,dmq360],axis=1)

dmx.tail(2)

roll120m  roll144m  roll180m  roll240m  roll300m  roll360m
2015-09-30  3.451056  3.737475  4.185031  4.925406  5.579070  6.273378
2015-12-31  3.382139  3.682372  4.108569  4.872263  5.511085  6.202494

In [7]:
## get libor-swap history:
ls = pd.read_csv("LibSwap_LongHist_30y_201512.csv",index_col=0)
ls.index = pd.to_datetime(ls.index)
#ls = ls[:'2014']
lsq=ls['1965':].resample('Q').mean()
## prepare to build ladders:
ld = lsq[['F1d','Prime']]
ld['DDA'] = 0.
mlst = [1,3,6,12,24,36,48,60,84,120,360]  # maturity list to be used
lcol = list(ls.columns)[1:-2]
lcol.remove('L12m')
mdct = dict(zip(mlst,lcol))
len(lsq), len(ld)

(204, 204)

In [8]:
mlst

[1, 3, 6, 12, 24, 36, 48, 60, 84, 120, 360]

In [9]:
## alternative code to process rolling averages:
ldm = ls[['F1d','Prime']]
m1 = ls[mdct[mlst[0]]] * 365/360
m3 = pd.rolling_mean(ls[mdct[mlst[1]]],mlst[1]) * 365/360
m6 = pd.rolling_mean(ls[mdct[mlst[2]]],mlst[2]) * 365/360
m12 = pd.rolling_mean(ls[mdct[mlst[3]]],mlst[3])
m24 = pd.rolling_mean(ls[mdct[mlst[4]]],mlst[4])
m36 = pd.rolling_mean(ls[mdct[mlst[5]]],mlst[5])
m48 = pd.rolling_mean(ls[mdct[mlst[6]]],mlst[6])
m60 = pd.rolling_mean(ls[mdct[mlst[7]]],mlst[7])
m84 = pd.rolling_mean(ls[mdct[mlst[8]]],mlst[8])
m120 = pd.rolling_mean(ls[mdct[mlst[9]]],mlst[9])
m360 = pd.rolling_mean(ls[mdct[mlst[10]]],mlst[10])
ldc = pd.concat([ldm,m1,m3,m6,m12,m24,m36,m48,m60,m84,m120,m360],axis=1)
ldc['F1d'] = ldc['F1d'] * 365/360 
ldc['Prime'] = ldc['Prime'] * 365/360 
#ldc.dropna(how='any',inplace=True)
ldx=ldc.resample('Q').mean()
#ldl=ldc.resample('Q',how='last')  # do not use
ldx.tail(4)

F1d     Prime       L1m       L3m       L6m       S1y  \
Date                                                                     
2015-03-31  0.111528  3.295139  0.174308  0.255746  0.349010  0.335556   
2015-06-30  0.125046  3.295139  0.186177  0.277658  0.388677  0.389167   
2015-09-30  0.138565  3.295139  0.198036  0.303004  0.446059  0.445556   
2015-12-31  0.162222  3.335694  0.255253  0.363004  0.530224  0.502778   

                 S2y       S3y       S4y       S5y       S7y      S10y  \
Date                                                                     
2015-03-31  0.568333  0.797593  1.133750  1.563389  2.478056  3.570583   
2015-06-30  0.625000  0.837222  1.110139  1.515000  2.401071  3.507111   
2015-09-30  0.675278  0.891296  1.122778  1.501056  2.316587  3.451056   
2015-12-31  0.725139  0.945278  1.143194  1.496944  2.244325  3.382139   

                S30y  
Date                  
2015-03-31  6.424487  
2015-06-30  6.344768  
2015-09-30  6.273378  
2015-12-31  6.202494

In [10]:
ldx = ldx['1965-12-31':]
len(ldx),len(dmx)

(201, 201)

In [11]:
ldx.head(2)

F1d     Prime       L1m       L3m       L6m       S1y  S2y  \
Date                                                                          
1965-12-31  4.224537  4.704444  4.611440  4.866537  4.880055  4.854038  NaN   
1966-03-31  4.619954  5.187731  5.033893  5.337432  5.256321  4.986816  NaN   

            S3y  S4y  S5y  S7y  S10y  S30y  
Date                                        
1965-12-31  NaN  NaN  NaN  NaN   NaN   NaN  
1966-03-31  NaN  NaN  NaN  NaN   NaN   NaN

In [12]:
dmx.head(2)

roll120m  roll144m  roll180m  roll240m  roll300m  roll360m
1965-12-31       NaN       NaN       NaN       NaN       NaN       NaN
1966-03-31       NaN       NaN       NaN       NaN       NaN       NaN

In [13]:
ldx[['S12y','S15y','S20y','S25y']] =dmx[['roll144m','roll180m','roll240m','roll300m']]

In [14]:
ldx = ldx[['F1d','L1m','L3m','L6m','S1y','S2y','S3y','S4y','S5y','S7y','S10y','S12y','S15y','S20y','S25y','S30y','Prime']]
ldx['DDA'] = 0
ldx.tail(2)

F1d       L1m       L3m       L6m       S1y       S2y  \
Date                                                                     
2015-09-30  0.138565  0.198036  0.303004  0.446059  0.445556  0.675278   
2015-12-31  0.162222  0.255253  0.363004  0.530224  0.502778  0.725139   

                 S3y       S4y       S5y       S7y      S10y      S12y  \
Date                                                                     
2015-09-30  0.891296  1.122778  1.501056  2.316587  3.451056  3.737475   
2015-12-31  0.945278  1.143194  1.496944  2.244325  3.382139  3.682372   

                S15y      S20y      S25y      S30y     Prime  DDA  
Date                                                               
2015-09-30  4.185031  4.925406  5.579070  6.273378  3.295139    0  
2015-12-31  4.108569  4.872263  5.511085  6.202494  3.335694    0

In [15]:
rlst = ['r'+str(x) for x in mlst+[144,180,240,300]]
rdct = dict(zip(lcol+['S12y','S15y','S20y','S25y'],rlst))
if 'L1m' in ldx.columns: ldx.rename(columns=rdct,inplace=True)
if 'F1d' in ldx.columns: ldx.rename(columns={'F1d':'r0'},inplace=True)
ldx = ldx[['r0','r1','r3','r6','r12','r24','r36','r48','r60','r84','r120','r144','r180','r240','r300','r360','Prime','DDA']]
ldx.tail()
#rdct

r0        r1        r3        r6       r12       r24  \
Date                                                                     
2014-12-31  0.101389  0.159309  0.237383  0.332449  0.300833  0.515694   
2015-03-31  0.111528  0.174308  0.255746  0.349010  0.335556  0.568333   
2015-06-30  0.125046  0.186177  0.277658  0.388677  0.389167  0.625000   
2015-09-30  0.138565  0.198036  0.303004  0.446059  0.445556  0.675278   
2015-12-31  0.162222  0.255253  0.363004  0.530224  0.502778  0.725139   

                 r36       r48       r60       r84      r120      r144  \
Date                                                                     
2014-12-31  0.761852  1.154375  1.616000  2.558770  3.632278  3.871096   
2015-03-31  0.797593  1.133750  1.563389  2.478056  3.570583  3.822723   
2015-06-30  0.837222  1.110139  1.515000  2.401071  3.507111  3.779909   
2015-09-30  0.891296  1.122778  1.501056  2.316587  3.451056  3.737475   
2015-12-31  0.945278  1.143194  1.496944  2.244325  3.382139  3.682372   

                r180      r240      r300      r360     Prime  DDA  
Date                                                               
2014-12-31  4.432026  5.114701  5.783254  6.505476  3.295139    0  
2015-03-31  4.349112  5.043419  5.716069  6.424487  3.295139    0  
2015-06-30  4.264183  4.979716  5.645333  6.344768  3.295139    0  
2015-09-30  4.185031  4.925406  5.579070  6.273378  3.295139    0  
2015-12-31  4.108569  4.872263  5.511085  6.202494  3.335694    0

In [16]:
mzlst = [0,1, 3, 6, 12, 24, 36, 48, 60, 84, 120]
rzlst = ['r0','r1','r3','r6','r12','r24','r36','r48','r60','r84','r120','Prime','DDA']
#if 0 not in mzlst: mzlst = [0] + mlst
#if 'r0' not in rzlst: rzlst = ['r0'] + rlst
ldxall = ldx['2001':].copy()  # keep ldxall as a reference table (for multiple runs)
ldxall.head(2)

r0        r1        r3        r6      r12       r24  \
Date                                                                    
2001-03-31  5.671019  5.584845  5.925261  6.289939  6.58615  6.441290   
2001-06-30  4.386759  4.350878  4.636757  5.156984  5.97003  6.360413   

                 r36       r48       r60       r84      r120      r144  \
Date                                                                     
2001-03-31  6.103628  6.175796  6.266308  6.514782  6.895917  7.321731   
2001-06-30  6.102206  6.095638  6.223161  6.481845  6.828667  7.247147   

                r180      r240      r300      r360     Prime  DDA  
Date                                                               
2001-03-31  7.729644  9.055592  9.238483  9.023238  8.743102    0  
2001-06-30  7.689809  8.956643  9.214836  9.019233  7.441944    0

In [17]:
mzlst

[0, 1, 3, 6, 12, 24, 36, 48, 60, 84, 120]

In [18]:
rzlst

['r0',
 'r1',
 'r3',
 'r6',
 'r12',
 'r24',
 'r36',
 'r48',
 'r60',
 'r84',
 'r120',
 'Prime',
 'DDA']

In [19]:
ldxall.tail(2)

r0        r1        r3        r6       r12       r24  \
Date                                                                     
2015-09-30  0.138565  0.198036  0.303004  0.446059  0.445556  0.675278   
2015-12-31  0.162222  0.255253  0.363004  0.530224  0.502778  0.725139   

                 r36       r48       r60       r84      r120      r144  \
Date                                                                     
2015-09-30  0.891296  1.122778  1.501056  2.316587  3.451056  3.737475   
2015-12-31  0.945278  1.143194  1.496944  2.244325  3.382139  3.682372   

                r180      r240      r300      r360     Prime  DDA  
Date                                                               
2015-09-30  4.185031  4.925406  5.579070  6.273378  3.295139    0  
2015-12-31  4.108569  4.872263  5.511085  6.202494  3.335694    0

In [20]:
grp='TestProd2'

In [21]:
## select time period: 2001-2014, 2001-2007, 2008-2014 (14y, 7y, 7y)
#r0 = '2001_2014'
#r0 = '2001_2007'
r0 = '2001_2010'
#r0 = '2005_2014'
ra = int(r0[0:4])
rb = int(r0[-4:])
r0, ra, rb

('2001_2010', 2001, 2010)

In [22]:
ldx = ldxall[str(ra):str(rb)]
ldx['r1.5'] = ldx['r1']*.5
ldx['r3.5'] = ldx['r3']*.5
ldx['r6.5'] = ldx['r6']*.5
ldx['r12.5'] = ldx['r12']*.5
ldx['r1.5_1'] = ldx['r1.5'].shift(1)
ldx['r3.5_1'] = ldx['r3.5'].shift(1)
ldx['r6.5_1'] = ldx['r6.5'].shift(1)
ldx['r12.5_1'] = ldx['r12.5'].shift(1)

In [23]:
ldx.head(2)

r0        r1        r3        r6      r12       r24  \
Date                                                                    
2001-03-31  5.671019  5.584845  5.925261  6.289939  6.58615  6.441290   
2001-06-30  4.386759  4.350878  4.636757  5.156984  5.97003  6.360413   

                 r36       r48       r60       r84    ...        Prime  DDA  \
Date                                                  ...                     
2001-03-31  6.103628  6.175796  6.266308  6.514782    ...     8.743102    0   
2001-06-30  6.102206  6.095638  6.223161  6.481845    ...     7.441944    0   

                r1.5      r3.5      r6.5     r12.5    r1.5_1    r3.5_1  \
Date                                                                     
2001-03-31  2.792422  2.962631  3.144970  3.293075       NaN       NaN   
2001-06-30  2.175439  2.318379  2.578492  2.985015  2.792422  2.962631   

             r6.5_1   r12.5_1  
Date                           
2001-03-31      NaN       NaN  
2001-06-30  3.14497  3.293075  

[2 rows x 26 columns]

In [24]:
ldx.tail(2)

r0        r1        r3        r6       r12       r24  \
Date                                                                     
2010-09-30  0.189259  0.292869  0.459604  0.600520  0.593889  1.486944   
2010-12-31  0.189259  0.260921  0.302902  0.574779  0.550556  1.222222   

                 r36       r48       r60       r84    ...        Prime  DDA  \
Date                                                  ...                     
2010-09-30  2.568241  3.456181  3.955611  4.241468    ...     3.295139    0   
2010-12-31  2.265370  3.216111  3.800000  4.170159    ...     3.295139    0   

                r1.5      r3.5     r6.5     r12.5    r1.5_1    r3.5_1  \
Date                                                                    
2010-09-30  0.146434  0.229802  0.30026  0.296944  0.159510  0.178901   
2010-12-31  0.130460  0.151451  0.28739  0.275278  0.146434  0.229802   

              r6.5_1   r12.5_1  
Date                            
2010-09-30  0.239778  0.322361  
2010-12-31  0.300260  0.296944  

[2 rows x 26 columns]

In [25]:

y9col=['r1.5','r1.5_1','r3.5','r3.5_1','r6.5','r6.5_1','r12.5','r12.5_1','DDA','r6','r12']

y9col

['r1.5',
 'r1.5_1',
 'r3.5',
 'r3.5_1',
 'r6.5',
 'r6.5_1',
 'r12.5',
 'r12.5_1',
 'DDA',
 'r6',
 'r12']

In [26]:
mx = np.arange(0,1.05,.05)  # mixes to analyze
yrcol=['grp','prod','mt1','mt2','mt3','mx1','mx2','dur','spd','sd']
yr = pd.DataFrame(columns=yrcol)

In [27]:
mzlst

[0, 1, 3, 6, 12, 24, 36, 48, 60, 84, 120]

In [28]:
rzlst

['r0',
 'r1',
 'r3',
 'r6',
 'r12',
 'r24',
 'r36',
 'r48',
 'r60',
 'r84',
 'r120',
 'Prime',
 'DDA']

In [29]:
d20y = pd.read_csv('durladder_last20y_adjmnts.csv',index_col=0)
d20y.index.names = ['mnth']
d20y.reset_index(drop=False,inplace=True)
d20dct = dict(zip(d20y['mnth'],d20y['dur']))
d20dct[0] = 0
d20dct[1] = 1/24
d20dct[3] = 3/24
d20dct[6] = 6/24
d20dct

{0: 0,
 1: 0.041666666666666664,
 3: 0.125,
 6: 0.25,
 12: 0.5372372286399999,
 24: 1.02023105292,
 36: 1.4886787413100002,
 48: 1.9409599189500002,
 60: 2.3760869494699999,
 72: 2.7937928221699999,
 84: 3.1937051845499997,
 96: 3.5757228953900002,
 108: 3.9395832749599999,
 120: 4.2887214980300001,
 144: 4.9492002901900003,
 180: 5.8382482804300002,
 240: 7.0755620997600008,
 300: 8.0893730982800012,
 360: 8.9561664775000001}

In [30]:
## process set of products in ycol:

for prd in y9col:
  ldx['prd'] = ldx[prd]
  print(prd+' started at:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
#  for i in [0,1]:
  for i in np.arange(0,len(mzlst)-2):
    mt1 = rzlst[i]
    for j in np.arange(i+1,len(mzlst)-1):
      mt2 = rzlst[j]
      for k in np.arange(j+1,len(mzlst)):
        mt3 = rzlst[k]
        for m1 in mx:
          for m2 in mx:
            if (m1==0)&(m2==0): sd=99999
            if (m1 + m2) <= 1:
              ldx['spd'] = ldx.apply(lambda x: x['prd'] - (m1*x[mt1] + m2*x[mt2] + (1-m1-m2)*x[mt3]),axis=1)
              if ldx['spd'].std() < sd:
                sd = ldx['spd'].std()
                avg = ldx['spd'].mean()
                dur = m1 * d20dct[mzlst[i]] + m2* d20dct[mzlst[j]] + (1-m1-m2)* d20dct[mzlst[k]]
                ydct = dict(zip(yrcol,[grp,prd,mt1,mt2,mt3,m1,m2,dur,avg,sd]))
            else:
              pass
        yr = yr.append(ydct,ignore_index=True)
    print(mt1, sd, dur, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


('r1.5 started at:', '2016-05-31 06:04:08')
('r0', 0.37265831870486793, 2.1443607490150001, '2016-05-31 06:08:59')
('r1', 0.3060964991975907, 2.1651940823483335, '2016-05-31 06:12:53')
('r3', 0.30864949611886755, 2.2068607490150001, '2016-05-31 06:15:54')
('r6', 0.35651725631112896, 2.0674246741135001, '2016-05-31 06:18:11')
('r12', 0.5066922929734583, 2.4129793633349998, '2016-05-31 06:19:49')
('r24', 0.8043277097992898, 2.8179007977305002, '2016-05-31 06:20:54')
('r36', 0.9684408194023504, 3.7287129466860005, '2016-05-31 06:21:33')
('r48', 0.9836023426059562, 4.2887214980300001, '2016-05-31 06:21:52')
('r60', 0.9836023426059562, 4.2887214980300001, '2016-05-31 06:21:59')
('r1.5_1 started at:', '2016-05-31 06:21:59')
('r0', 0.4461658753518736, 2.1443607490150001, '2016-05-31 06:26:52')
('r1', 0.38984441566364325, 2.1651940823483335, '2016-05-31 06:30:48')
('r3', 0.31465220093363677, 2.2068607490150001, '2016-05-31 06:33:50')
('r6', 0.24843666462410557, 2.0674246741135001, '2016-05-31 

In [31]:
yr.head()

grp  prod mt1 mt2  mt3   mx1  mx2       dur       spd        sd
0  TestProd2  r1.5  r0  r1   r3  0.25  0.1  0.085417 -1.342089  0.907300
1  TestProd2  r1.5  r0  r1   r6  0.20  0.2  0.158333 -1.462785  0.875889
2  TestProd2  r1.5  r0  r1  r12  0.00  0.5  0.289452 -1.552709  0.856415
3  TestProd2  r1.5  r0  r1  r24  0.05  0.5  0.479937 -1.792312  0.693689
4  TestProd2  r1.5  r0  r1  r36  0.00  0.6  0.620471 -1.938803  0.511192

In [32]:
yrn = yr.ix[yr.groupby(['prod']).apply(lambda x: x['sd'].idxmin())]
yrn

grp     prod mt1  mt2   mt3   mx1   mx2       dur       spd  \
1343  TestProd2      DDA  r0   r6  r120  0.00  0.05  4.086785 -5.693406   
59    TestProd2     r1.5  r1   r6  r120  0.40  0.15  1.984091 -2.809872   
291   TestProd2   r1.5_1  r6  r48  r120  0.55  0.10  1.832649 -2.734470   
1652  TestProd2      r12  r0   r1   r12  0.00  0.00  0.537237  0.000000   
1116  TestProd2    r12.5  r6  r48  r120  0.50  0.20  1.799808 -2.574428   
1271  TestProd2  r12.5_1  r6  r24   r48  0.45  0.05  1.133992 -2.132824   
417   TestProd2     r3.5  r3   r6  r120  0.35  0.20  2.023675 -2.803781   
621   TestProd2   r3.5_1  r6  r48  r120  0.55  0.25  1.480484 -2.457996   
1486  TestProd2       r6  r0   r1    r6  0.00  0.00  0.250000  0.000000   
683   TestProd2     r6.5  r0   r6  r120  0.00  0.55  2.067425 -2.773957   
949   TestProd2   r6.5_1  r6  r48   r60  0.55  0.35  1.054445 -2.127127   

            sd  
1343  0.604397  
59    0.300484  
291   0.243864  
1652  0.000000  
1116  0.240862  
1271  0.237223  
417   0.300824  
621   0.260405  
1486  0.000000  
683   0.304578  
949   0.294148

In [33]:
yr.to_csv('yldreg_3mat_'+grp+'_'+r0+'.csv')
yrn.to_csv('yldreg_3matbest_'+grp+'_'+r0+'.csv')

In [34]:
'yldreg_3matbest_'+grp+'_'+r0+'.csv'

'yldreg_3matbest_TestProd2_2001_2010.csv'

In [35]:
len(yr), len(yrn)

(1815, 11)